In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys

import re
import pandas as pd
import subprocess
import time
from tqdm import tqdm
import concurrent
import threading
import os

## GitHub APIs

In [16]:
# GitHub API search URL with pagination parameters
GITHUB_TOKEN = 'x'
HEADERS = {'Authorization': f'token {GITHUB_TOKEN}'}

# GitHub Selenium 

In [17]:
#chrome_options = Options()
#chrome_options.add_argument("--headless")
#driver = webdriver.Chrome(options=chrome_options)
un = "vincent-stadler"
pw = "x"
start = 0
lock = threading.Lock()

### Note about sparse cross entropy:
There are a number of situations to use scce, including:
- when your classes are mutually exclusive, i.e. you don’t care at all about other close-enough predictions,
- the number of categories is large to the prediction output becomes overwhelming.

<i>https://fmorenovr.medium.com/sparse-categorical-cross-entropy-vs-categorical-cross-entropy-ea01d0392d28</i>

In [8]:
import itertools

base_url = "https://github.com/search?q="

base_keywords = [['tensorflow'], ['"step - loss:"'], ['"Epoch ' + str(i * 10)  + '"' for i in range(1, 10)], ['path:.ipynb']]
base_keywords = [" AND ".join((a,b,c)) for a in base_keywords[0] for b in base_keywords[1] for c in base_keywords[2]]
architectures  = ['transformer', 'sequential cnn']
loss_functions = ['(sparse_categorical_crossentropy OR SparseCategoricalCrossentropy)', '(categorical_crossentropy OR CategoricalCrossentropy)']
datasets = ['fashion_mnist', 'cifar10']

combinations = list(itertools.product(*[base_keywords, architectures, loss_functions, datasets]))
queries = []
for combination in combinations:
    query = base_url + ' AND '.join(combination) + '&type=code'
    queries.append(query)


queries = [query + "&p={}".format(j) for query in queries for j in range(1,6)]
(queries)

['https://github.com/search?q=tensorflow AND "step - loss:" AND "Epoch 10" AND transformer AND (sparse_categorical_crossentropy OR SparseCategoricalCrossentropy) AND fashion_mnist&type=code&p=1',
 'https://github.com/search?q=tensorflow AND "step - loss:" AND "Epoch 10" AND transformer AND (sparse_categorical_crossentropy OR SparseCategoricalCrossentropy) AND fashion_mnist&type=code&p=2',
 'https://github.com/search?q=tensorflow AND "step - loss:" AND "Epoch 10" AND transformer AND (sparse_categorical_crossentropy OR SparseCategoricalCrossentropy) AND fashion_mnist&type=code&p=3',
 'https://github.com/search?q=tensorflow AND "step - loss:" AND "Epoch 10" AND transformer AND (sparse_categorical_crossentropy OR SparseCategoricalCrossentropy) AND fashion_mnist&type=code&p=4',
 'https://github.com/search?q=tensorflow AND "step - loss:" AND "Epoch 10" AND transformer AND (sparse_categorical_crossentropy OR SparseCategoricalCrossentropy) AND fashion_mnist&type=code&p=5',
 'https://github.com

In [13]:
os.chdir(r"C:\Users\daV\Documents\ZHAW\HS 2024\dPoDL\dPoDL\experiments\filtered-search")
os.makedirs("transformers/mnist/sparse-categorical", exist_ok=True)
os.makedirs("cnns/mnist/sparse-categorical", exist_ok=True)
os.makedirs("transformers/mnist/categorical", exist_ok=True)
os.makedirs("cnns/mnist/categorical", exist_ok=True)

os.makedirs("transformers/cifar10/sparse-categorical", exist_ok=True)
os.makedirs("cnns/cifar10/sparse-categorical", exist_ok=True)
os.makedirs("transformers/cifar10/categorical", exist_ok=True)
os.makedirs("cnns/cifar10/categorical", exist_ok=True)

def save_path(u):
    model_type = "transformers" if "transformer" in u else "cnns" if "sequential cnn" in u else ""
    dataset = "mnist" if "mnist" in u else "cifar10" if "cifar10" in u else ""
    loss_type = "sparse-categorical" if "sparse_categorical_crossentropy" in u else "categorical" if "categorical_crossentropy" in u else ""

    if model_type and dataset and loss_type:
        return f"{model_type}/{dataset}/{loss_type}"

print(queries[300])
print(save_path(queries[300]))

https://github.com/search?q=tensorflow AND "step - loss:" AND "Epoch 80" AND sequential cnn AND (sparse_categorical_crossentropy OR SparseCategoricalCrossentropy) AND fashion_mnist&type=code&p=1
cnns/mnist/sparse-categorical


In [14]:
def login(d):
    d.get("https://github.com/login")
    wait = WebDriverWait(d, 10)
    username_field = wait.until(EC.presence_of_element_located((By.ID, "login_field")))
    username_field.send_keys(un)
    # Input the password
    password_field = d.find_element(By.ID, "password")
    password_field.send_keys(pw)
    password_field.send_keys(Keys.RETURN)
    wait.until(EC.url_changes("https://github.com/login"))

def find_links(soup, text_file):
    l = []
    items = soup.find_all("div", "bmcJak")
    if len(items) > 0:
        for item in items:
            try:
                link = item.find("a", "gwACeB")["href"]
                link = ("https://raw.githubusercontent.com" + link).replace("/blob", "")
                l.append(link)
            except (KeyError, ValueError) as e:
                print(e)
        with open(text_file, 'a') as f:
            for item in l:
                f.write(item + '\n') 

def fetch_links(d, url, text_file):
    d.get(url)
    WebDriverWait(d, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'Box-sc-g0xbh4-0'))
    )
    soup = bs(d.page_source, 'html.parser')
    find_links(soup, text_file)

In [18]:
active_driver = webdriver.Chrome()
login(active_driver)

In [20]:
for url in queries:
    fetch_links(active_driver, url, os.path.join(save_path(url), "urls.txt"))